In [2698]:
import cv2
import numpy as np
import copy
import time

### 2-1

In [2699]:
def img_show(img,title=""):
    img=((img-img.min())/(img.max()-img.min())* 255.).astype(np.uint8) 
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [2700]:
def wrappedImgShow(bg,img):
    bgImg=copy.deepcopy(bg)
    h,w=bg.shape
    for i in range(h):
        for j in range(w):
            if img[i,j]!=0:
                bgImg[i,j]=img[i,j]
    img_show(bgImg)

In [2701]:
def stichedImgShow(bg,img):
    bgImg=copy.deepcopy(bg)
    h,w=bg.shape
    for i in range(h):
        for j in range(w):
            if img[i,j]!=0 and bgImg[i,j]==0:
                bgImg[i,j]=img[i,j]
    img_show(bgImg)

In [2702]:
def hammingDistance(des1,des2):
    newDistanceList=[]
    for i,d1 in enumerate(des1):
        desList=[]
        for j, d2 in enumerate(des2):
            desList.append(np.count_nonzero((d1[:,None] & (1 << np.arange(8))) != (d2[:,None] & (1 << np.arange(8)))))
        mini=min(desList)
        newDistanceList.append([i,desList.index(mini),mini])
    return newDistanceList

In [2703]:
def getDMatch(matches):
    DMatchList=[]
    for match in matches:
        dMatch=cv2.DMatch()
        dMatch.queryIdx,dMatch.trainIdx,dMatch.distance=map(int,match)
        DMatchList.append(dMatch)
    return DMatchList

In [2704]:
def getPositionMatrices(img1,img2,N):
    
    orb1=cv2.ORB_create()
    orb2=cv2.ORB_create()
    
    kp1,des1=orb1.detectAndCompute(img1,None)
    kp2,des2=orb2.detectAndCompute(img2,None)
    matches=sorted(hammingDistance(des1,des2),key=lambda x:x[2])
    
    topMatches=np.array(matches[:N])
    dst=cv2.drawMatches(img1,kp1,img2,kp2,getDMatch(topMatches),None,flags=2)
    img_show(dst)
    
    srcP,destP=[],[]
    for p1,p2 in zip(topMatches[:,0],topMatches[:,1]):
        destP.append([kp1[p1].pt[0],kp1[p1].pt[1]])
        srcP.append([kp2[p2].pt[0],kp2[p2].pt[1]])
        
    return np.array(srcP),np.array(destP)


### 2-2

In [2705]:
def getNormalizingMatrix(matrix):
    ##Todo mean subtraction
    mean=np.mean(matrix)
    subtractingMatrix=np.array([[1,0,-mean],[0,1,-mean],[0,0,1]])
    ##Todo scaling
    subtractedMatrix=subtractingMatrix@matrix.T
    maxi,mini=np.max(subtractedMatrix),np.min(subtractedMatrix)
    scalingValue=1/(maxi-mini)
    scalingMatrix1=np.array([[1,0,-mini],[0,1,-mini],[0,0,1]])
    scalingMatrix2=np.array([[scalingValue,0,0],[0,scalingValue,0],[0,0,1]]) #x,y 각각 1 => 최대 sqrt(2)
    return scalingMatrix2@scalingMatrix1@subtractingMatrix

In [2706]:
# def getNormalizingMatrix(matrix):
#     ##Todo mean subtraction
#     mean=np.mean(matrix,axis=0)
#     subtractingMatrix=np.array([[1,0,-mean[0]],[0,1,-mean[1]],[0,0,1]])
#     ##Todo scaling
#     subtractedMatrix=subtractingMatrix@matrix.T
#     print(subtractedMatrix.shape)
#     maxi,mini=np.max(subtractedMatrix[:2],axis=1),np.min(subtractedMatrix[:2],axis=1)
#     print(maxi,mini)
#     # maxi,mini=np.max(subtractedMatrix),np.min(subtractedMatrix)
#     scalingValue=1/(maxi-mini)
#     # print("max",mini,maxi,scalingValue)
#     scalingMatrix1=np.array([[1,0,-mini[0]],[0,1,-mini[1]],[0,0,1]])
#     # scaledMatrix=scalingMatrix1@matrix
#     # print(matrix)
#     if scalingValue[0]==0:
#         print(scalingValue[0])
#         scalingValue[0]=1
#     if scalingValue[1]==0:
#         print(scalingValue[1])
#         scalingValue[1]=1
#     scalingMatrix2=np.array([[scalingValue[0],0,0],[0,scalingValue[1],0],[0,0,1]]) #x,y 각각 1 => 최대 sqrt(2)
#     # print(scalingMatrix2.shape,scalingMatrix1.shape,subtractingMatrix.shape)
#     return scalingMatrix2@scalingMatrix1@subtractingMatrix

In [2707]:
# def getNormalizingMatrix(matrix):
#     ##Todo mean subtraction
#     mean=np.mean(matrix,axis=0)
#     subtractingMatrix=np.array([[1,0,-mean[0]],[0,1,-mean[1]],[0,0,1]])

#     ##Todo scaling
#     matrix=subtractingMatrix@matrix.T
#     mini=np.min(matrix)
#     scalingMatrix1=np.array([[1,0,-mini],[0,1,-mini],[0,0,1]])

#     matrix=scalingMatrix1@matrix
#     maxi=0
#     X,Y=0,0
#     for i,data in enumerate(matrix.T):
#         x,y,_=data
#         tmp=x**2+y**2
#         if maxi<=tmp:
#             maxi=tmp
#             X,Y=x,y
#     scalingX=1 if X==0 else X
#     scalingY=1 if Y==0 else Y
#     scaling=1/(scalingX+scalingY)
#     scalingMatrix2=np.array([[scaling,0,0],[0,scaling,0],[0,0,1]]) #x,y 각각 1 => sqrt(2)
#     return scalingMatrix2@scalingMatrix1@subtractingMatrix

In [2708]:
def getAMatrix(srcP,destP):
    A=[]
    for p,pDot in zip(srcP,destP):
        x,y=p; dx,dy=pDot
        Ai=np.array([[-x,-y,-1,0,0,0,x*dx,y*dx,dx],
                     [0,0,0,-x,-y,-1,x*dy,y*dy,dy]])
        if len(A)==0:  A=Ai
        else:  A=np.vstack([A,Ai])
    # print(A.shape) #2N*9
    return np.array(A)

In [2709]:
def compute_homography(srcP,destP):
    N=len(srcP)
    srcP1=np.append(srcP,np.ones((N,1)),axis=1)
    srcPMatrix=getNormalizingMatrix(srcP1)
    normalizedSrcP=srcPMatrix@srcP1.T
    if 0 not in normalizedSrcP[-1]:
        normalizedSrcP/=normalizedSrcP[-1]
        
    destP1=np.append(destP,np.ones((N,1)),axis=1)
    destPMatrix=getNormalizingMatrix(destP1)
    normalizedDestP=destPMatrix@destP1.T
    if 0 not in normalizedDestP[-1]:
        normalizedDestP/=normalizedDestP[-1]
    
    A=getAMatrix(normalizedSrcP[:2].T,normalizedDestP[:2].T)
    
    U,s,V=np.linalg.svd(A)
    h=np.reshape(V[-1],(3,3))
    H=np.linalg.inv(destPMatrix)@h@srcPMatrix
    return H/H[-1,-1]

### 2-3

In [2710]:
def compute_homography_ransac(srcP,destP,th=0.8):
    maxi,inlierList=0,[]
    N,num=len(srcP),4
    srcP=np.append(srcP,np.ones((N,1)),axis=1) #(30,3)
    for i in range(5000):
        idxList=np.random.choice(N,num,replace=False)
        
        randomChoiceSrcP,randomChoiceDestP=srcP[idxList,:2], destP[idxList]
        H=compute_homography(randomChoiceSrcP,randomChoiceDestP)
        sampleDestP=H@srcP.T
        
        if 0 not in sampleDestP[-1]:
            sampleDestP/=sampleDestP[-1]
        else:
            continue
        sampleDestP=sampleDestP[:2].T

        tmp = []
        for j,point in enumerate(zip(destP,sampleDestP)):
            dP,sP=point
            if (np.abs(dP[0]-sP[0])<=th) and (np.abs(dP[1]-sP[1])<=th):
                tmp.append(j)

        tmpLen=len(tmp)
        if tmpLen>maxi:
            inlierList=tmp
            maxi=tmpLen
    return compute_homography(srcP[inlierList,:2],destP[inlierList])

### 2-4

In [2713]:
np.random.seed(1000)
img1=cv2.imread('cv_desk.png',cv2.IMREAD_GRAYSCALE)
img2=cv2.imread('cv_cover.jpg',cv2.IMREAD_GRAYSCALE)

N=30
srcP,destP=getPositionMatrices(img1,img2,N)
H=compute_homography(srcP,destP)
transformed_img=cv2.warpPerspective(img2,H,img1.shape[::-1])
wrappedImgShow(img1,transformed_img)
# print("normal\n",H)

# H1,status=cv2.findHomography(srcP,destP)
# transformed_img = cv2.warpPerspective(img2, H1, img1.shape[::-1])
# wrappedImgShow(img1,transformed_img)
# print("cv2\n",H1)

start=time.time()
H2=compute_homography_ransac(srcP,destP,0.8)
print(f"ransac: {time.time()-start}")
transformed_img = cv2.warpPerspective(img2, H2, img1.shape[::-1])
wrappedImgShow(img1,transformed_img)
# print("ransac\n",H2)

img3=cv2.imread('hp_cover.jpg',cv2.IMREAD_GRAYSCALE)
transformed_img=cv2.warpPerspective(cv2.resize(img3,img2.shape[::-1]),H2,img1.shape[::-1])
wrappedImgShow(img1,transformed_img)


normal
 [[ 4.95216125e-01 -1.92762828e+00  3.43683201e+02]
 [ 1.76175657e-01 -1.68218888e+00  3.20755088e+02]
 [ 6.59297811e-04 -4.99729901e-03  1.00000000e+00]]
ransac: 2.531964063644409
ransac
 [[ 6.48508480e-01 -3.92664699e-01  2.50033533e+02]
 [-5.20476732e-02  1.67485403e-01  1.99693021e+02]
 [-1.41315099e-04 -1.00563658e-03  1.00000000e+00]]


### 2-5

In [2714]:
img4=cv2.imread('diamondhead-10.png',cv2.IMREAD_GRAYSCALE)
img5=cv2.imread('diamondhead-11.png',cv2.IMREAD_GRAYSCALE)

N=20
srcP,destP=getPositionMatrices(img4,img5,N)
H3=compute_homography_ransac(srcP,destP,0.8)
transformedSize=(img4.shape[1]+int(destP[0,0]-srcP[0,0]),img4.shape[0])
wrappedImg = cv2.warpPerspective(img5,H3,transformedSize) 
bgSize=(img4.shape[0],int(destP[0,0]-srcP[0,0]))
bgImg=np.column_stack((img4,np.zeros(bgSize)))
stichedImgShow(bgImg,wrappedImg)

In [2715]:
def gradationImgShow(stitched,warpped,end):
    n=100
    start=end-n
    for i in range(start,end):
        p=(i-start)/n
        stitched[:,i]=warpped[:,i]*p+stitched[:,i]*(1-p)
    img_show(stitched)